# DataFest 2025: Commercial Real Estate Leasing Analysis

## Uncovering Post-Pandemic Trends in the Office Market

**Challenge**: Identify trends in the commercial real estate marketplace that help Savills advise clients on where, when, and how to locate their offices.

**Key Questions**:
1. How did the COVID-19 pandemic reshape office leasing patterns?
2. Which industries are driving the recovery vs. retreating?
3. Is there a flight to quality (Class A) or flight to value (Class O)?
4. Are tenants choosing CBD or suburban locations post-pandemic?
5. How have lease sizes changed (fragmentation vs. consolidation)?

---

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Style settings
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

# Load data
DATA_PATH = Path('../data/raw/Leases.csv')
if not DATA_PATH.exists():
    DATA_PATH = Path('../../2025 Data Files/Leases.csv')

df = pd.read_csv(DATA_PATH)
print(f"Loaded {len(df):,} lease records")
print(f"Date range: {df['year'].min()} to {df['year'].max()}")
print(f"Markets: {df['market'].nunique()}")

## 1. Data Overview & Cleaning

In [ ]:
# Basic info
print("=" * 60)
print("DATASET OVERVIEW")
print("=" * 60)
print(f"\nShape: {df.shape[0]:,} rows x {df.shape[1]} columns")
print(f"\nColumn types:")
print(df.dtypes.value_counts())

# Missing data
missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(1)
missing_df = pd.DataFrame({'Missing': missing, 'Pct': missing_pct})
print(f"\nColumns with >20% missing:")
print(missing_df[missing_df['Pct'] > 20].sort_values('Pct', ascending=False))

In [ ]:
# Create time period categories
def categorize_period(year, quarter):
    if year < 2020:
        return 'Pre-Pandemic (2018-2019)'
    elif year == 2020:
        return 'Pandemic Year 1 (2020)'
    elif year == 2021:
        return 'Pandemic Year 2 (2021)'
    elif year == 2022:
        return 'Recovery Year 1 (2022)'
    elif year == 2023:
        return 'Recovery Year 2 (2023)'
    else:
        return 'Current (2024+)'

df['period'] = df.apply(lambda x: categorize_period(x['year'], x['quarter']), axis=1)
df['year_quarter'] = df['year'].astype(str) + '-' + df['quarter']

# Create date column for time series
quarter_map = {'Q1': '01', 'Q2': '04', 'Q3': '07', 'Q4': '10'}
df['date'] = pd.to_datetime(df['year'].astype(str) + '-' + df['quarter'].map(quarter_map) + '-01')

print("Time periods:")
print(df['period'].value_counts().sort_index())

---

## 2. Pandemic Impact Analysis

**Key Insight**: The pandemic created a structural break in office leasing. Let's quantify the damage and recovery.

In [ ]:
# Quarterly leasing volume
quarterly = df.groupby('date').agg({
    'leasedSF': ['sum', 'count', 'mean', 'median'],
    'company_name': 'nunique'
}).reset_index()
quarterly.columns = ['date', 'total_sf', 'num_leases', 'avg_sf', 'median_sf', 'unique_tenants']

# Plot leasing volume over time
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Total SF
ax1 = axes[0, 0]
ax1.plot(quarterly['date'], quarterly['total_sf'] / 1e6, 'b-', linewidth=2)
ax1.axvline(pd.Timestamp('2020-03-01'), color='red', linestyle='--', alpha=0.7, label='COVID-19')
ax1.fill_between(quarterly['date'], 0, quarterly['total_sf'] / 1e6, alpha=0.3)
ax1.set_ylabel('Total SF Leased (Millions)')
ax1.set_title('Total Leased Square Footage by Quarter')
ax1.legend()

# Number of leases
ax2 = axes[0, 1]
ax2.plot(quarterly['date'], quarterly['num_leases'], 'g-', linewidth=2)
ax2.axvline(pd.Timestamp('2020-03-01'), color='red', linestyle='--', alpha=0.7)
ax2.set_ylabel('Number of Leases')
ax2.set_title('Number of Lease Transactions by Quarter')

# Average lease size
ax3 = axes[1, 0]
ax3.plot(quarterly['date'], quarterly['avg_sf'], 'orange', linewidth=2, label='Mean')
ax3.plot(quarterly['date'], quarterly['median_sf'], 'purple', linewidth=2, label='Median')
ax3.axvline(pd.Timestamp('2020-03-01'), color='red', linestyle='--', alpha=0.7)
ax3.set_ylabel('Square Footage')
ax3.set_title('Average vs Median Lease Size')
ax3.legend()

# Unique tenants
ax4 = axes[1, 1]
ax4.bar(quarterly['date'], quarterly['unique_tenants'], width=60, color='teal', alpha=0.7)
ax4.axvline(pd.Timestamp('2020-03-01'), color='red', linestyle='--', alpha=0.7)
ax4.set_ylabel('Unique Tenants')
ax4.set_title('Unique Companies Signing Leases')

plt.tight_layout()
plt.savefig('../outputs/pandemic_impact_timeseries.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Calculate pandemic impact metrics
pre_pandemic = df[df['year'].isin([2018, 2019])]
pandemic = df[df['year'].isin([2020, 2021])]
recovery = df[df['year'].isin([2022, 2023, 2024])]

print("=" * 60)
print("PANDEMIC IMPACT SUMMARY")
print("=" * 60)

metrics = {
    'Pre-Pandemic (2018-19)': pre_pandemic,
    'Pandemic (2020-21)': pandemic,
    'Recovery (2022+)': recovery
}

for period, data in metrics.items():
    n_years = data['year'].nunique()
    print(f"\n{period}:")
    print(f"  Total leases: {len(data):,} ({len(data)/n_years:,.0f}/year)")
    print(f"  Total SF: {data['leasedSF'].sum()/1e6:.1f}M ({data['leasedSF'].sum()/n_years/1e6:.1f}M/year)")
    print(f"  Median lease: {data['leasedSF'].median():,.0f} SF")
    print(f"  Unique tenants: {data['company_name'].nunique():,}")

# Calculate % changes
print("\n" + "=" * 60)
print("% CHANGE FROM PRE-PANDEMIC BASELINE")
print("=" * 60)

pre_annual_sf = pre_pandemic['leasedSF'].sum() / 2
pandemic_annual_sf = pandemic['leasedSF'].sum() / 2
recovery_annual_sf = recovery['leasedSF'].sum() / recovery['year'].nunique()

print(f"\nPandemic vs Pre-Pandemic: {(pandemic_annual_sf/pre_annual_sf - 1)*100:+.1f}%")
print(f"Recovery vs Pre-Pandemic: {(recovery_annual_sf/pre_annual_sf - 1)*100:+.1f}%")

---

## 3. Industry Analysis: Winners & Losers

**Key Question**: Which industries are driving demand post-pandemic?

In [ ]:
# Industry leasing by period
industry_period = df.groupby(['internal_industry', 'period']).agg({
    'leasedSF': 'sum',
    'company_name': 'count'
}).reset_index()
industry_period.columns = ['industry', 'period', 'total_sf', 'num_leases']

# Top industries overall
top_industries = df.groupby('internal_industry')['leasedSF'].sum().nlargest(10).index.tolist()

# Filter to top industries
industry_top = industry_period[industry_period['industry'].isin(top_industries)]

# Pivot for comparison
industry_pivot = industry_top.pivot(index='industry', columns='period', values='total_sf').fillna(0)

# Reorder columns chronologically
period_order = ['Pre-Pandemic (2018-2019)', 'Pandemic Year 1 (2020)', 'Pandemic Year 2 (2021)', 
                'Recovery Year 1 (2022)', 'Recovery Year 2 (2023)', 'Current (2024+)']
industry_pivot = industry_pivot[[c for c in period_order if c in industry_pivot.columns]]

# Calculate change
if 'Pre-Pandemic (2018-2019)' in industry_pivot.columns and 'Recovery Year 2 (2023)' in industry_pivot.columns:
    industry_pivot['change_pct'] = (
        (industry_pivot['Recovery Year 2 (2023)'] / (industry_pivot['Pre-Pandemic (2018-2019)']/2)) - 1
    ) * 100
    industry_pivot = industry_pivot.sort_values('change_pct', ascending=False)

print("Industry Leasing by Period (SF in Millions):")
display_df = industry_pivot.copy()
for col in display_df.columns[:-1]:
    display_df[col] = (display_df[col] / 1e6).round(1)
display_df['change_pct'] = display_df['change_pct'].round(1)
print(display_df)

In [ ]:
# Visualize industry trends
fig, axes = plt.subplots(1, 2, figsize=(16, 7))

# Stacked bar chart
ax1 = axes[0]
industry_top_pivot = df[df['internal_industry'].isin(top_industries[:8])].pivot_table(
    index='year', 
    columns='internal_industry', 
    values='leasedSF', 
    aggfunc='sum'
).fillna(0)

(industry_top_pivot / 1e6).plot(kind='bar', stacked=True, ax=ax1, colormap='tab10')
ax1.set_ylabel('Total SF Leased (Millions)')
ax1.set_xlabel('Year')
ax1.set_title('Industry Leasing Volume by Year')
ax1.legend(title='Industry', bbox_to_anchor=(1.02, 1), loc='upper left', fontsize=8)
ax1.axvline(1.5, color='red', linestyle='--', alpha=0.7)  # COVID line
plt.setp(ax1.xaxis.get_majorticklabels(), rotation=0)

# Winners vs Losers
ax2 = axes[1]
if 'change_pct' in industry_pivot.columns:
    colors = ['green' if x > 0 else 'red' for x in industry_pivot['change_pct']]
    industry_pivot['change_pct'].plot(kind='barh', ax=ax2, color=colors)
    ax2.axvline(0, color='black', linewidth=0.5)
    ax2.set_xlabel('% Change from Pre-Pandemic (annualized)')
    ax2.set_title('Industry Recovery: 2023 vs 2018-2019 Average')

plt.tight_layout()
plt.savefig('../outputs/industry_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

---

## 4. Flight to Quality vs Flight to Value

**Hypothesis**: Post-pandemic, companies are choosing premium (Class A) buildings to attract employees back to office.

In [ ]:
# Class A vs O breakdown by year
class_year = df.groupby(['year', 'internal_class']).agg({
    'leasedSF': ['sum', 'count', 'mean']
}).reset_index()
class_year.columns = ['year', 'class', 'total_sf', 'num_leases', 'avg_sf']

# Calculate Class A share
yearly_totals = class_year.groupby('year')['total_sf'].sum().reset_index()
yearly_totals.columns = ['year', 'yearly_total']

class_year = class_year.merge(yearly_totals, on='year')
class_year['share'] = class_year['total_sf'] / class_year['yearly_total'] * 100

class_a = class_year[class_year['class'] == 'A']

# Visualize
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Class A share over time
ax1 = axes[0]
ax1.plot(class_a['year'], class_a['share'], 'b-o', linewidth=2, markersize=8)
ax1.axhline(class_a['share'].mean(), color='gray', linestyle='--', alpha=0.7, label=f"Avg: {class_a['share'].mean():.1f}%")
ax1.axvline(2020, color='red', linestyle='--', alpha=0.5)
ax1.set_xlabel('Year')
ax1.set_ylabel('Class A Share (%)')
ax1.set_title('Class A Share of Total Leased SF')
ax1.legend()
ax1.set_ylim(0, 100)

# Total SF by class
ax2 = axes[1]
class_pivot = class_year.pivot(index='year', columns='class', values='total_sf').fillna(0)
(class_pivot / 1e6).plot(kind='bar', ax=ax2, color=['#2ecc71', '#3498db'])
ax2.set_ylabel('Total SF (Millions)')
ax2.set_title('Leasing Volume by Building Class')
ax2.legend(title='Class')
plt.setp(ax2.xaxis.get_majorticklabels(), rotation=0)

# Average lease size by class
ax3 = axes[2]
class_pivot_avg = class_year.pivot(index='year', columns='class', values='avg_sf').fillna(0)
class_pivot_avg.plot(kind='line', ax=ax3, marker='o', linewidth=2)
ax3.set_ylabel('Average Lease Size (SF)')
ax3.set_title('Average Lease Size by Building Class')
ax3.legend(title='Class')

plt.tight_layout()
plt.savefig('../outputs/flight_to_quality.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nKey Finding:")
pre_a = class_a[class_a['year'].isin([2018, 2019])]['share'].mean()
post_a = class_a[class_a['year'].isin([2022, 2023, 2024])]['share'].mean()
print(f"Class A share pre-pandemic: {pre_a:.1f}%")
print(f"Class A share post-pandemic: {post_a:.1f}%")
print(f"Change: {post_a - pre_a:+.1f} percentage points")

---

## 5. CBD vs Suburban: The Great Migration?

**Hypothesis**: Post-pandemic, tenants are moving to suburban locations for larger, cheaper space.

In [ ]:
# CBD vs Suburban analysis
location_year = df.groupby(['year', 'CBD_suburban']).agg({
    'leasedSF': ['sum', 'count', 'mean']
}).reset_index()
location_year.columns = ['year', 'location', 'total_sf', 'num_leases', 'avg_sf']

# Calculate CBD share
yearly_loc = location_year.groupby('year')['total_sf'].sum().reset_index()
yearly_loc.columns = ['year', 'yearly_total']
location_year = location_year.merge(yearly_loc, on='year')
location_year['share'] = location_year['total_sf'] / location_year['yearly_total'] * 100

# Visualize
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Share over time
ax1 = axes[0]
for loc in location_year['location'].unique():
    data = location_year[location_year['location'] == loc]
    ax1.plot(data['year'], data['share'], '-o', linewidth=2, markersize=8, label=loc)
ax1.axvline(2020, color='red', linestyle='--', alpha=0.5)
ax1.set_xlabel('Year')
ax1.set_ylabel('Share of Total SF (%)')
ax1.set_title('CBD vs Suburban Share Over Time')
ax1.legend()

# Volume comparison
ax2 = axes[1]
loc_pivot = location_year.pivot(index='year', columns='location', values='total_sf').fillna(0)
(loc_pivot / 1e6).plot(kind='bar', ax=ax2, color=['#e74c3c', '#3498db'])
ax2.set_ylabel('Total SF (Millions)')
ax2.set_title('Leasing Volume: CBD vs Suburban')
plt.setp(ax2.xaxis.get_majorticklabels(), rotation=0)

# Average size comparison
ax3 = axes[2]
loc_avg = location_year.pivot(index='year', columns='location', values='avg_sf').fillna(0)
loc_avg.plot(kind='line', ax=ax3, marker='o', linewidth=2)
ax3.set_ylabel('Average Lease Size (SF)')
ax3.set_title('Average Lease Size by Location Type')

plt.tight_layout()
plt.savefig('../outputs/cbd_vs_suburban.png', dpi=150, bbox_inches='tight')
plt.show()

# Summary stats
print("\nCBD vs Suburban Summary:")
for loc in ['CBD', 'Suburban']:
    pre = location_year[(location_year['location'] == loc) & (location_year['year'].isin([2018, 2019]))]['share'].mean()
    post = location_year[(location_year['location'] == loc) & (location_year['year'].isin([2022, 2023, 2024]))]['share'].mean()
    print(f"{loc}: {pre:.1f}% (pre) → {post:.1f}% (post) | Change: {post-pre:+.1f}pp")

---

## 6. Lease Size Fragmentation Analysis

**Key Question**: Are companies taking smaller, more flexible spaces post-pandemic?

In [ ]:
# Lease size categories
def size_category(sf):
    if sf < 2500:
        return '1. Micro (<2.5K)'
    elif sf < 5000:
        return '2. Small (2.5-5K)'
    elif sf < 10000:
        return '3. Medium (5-10K)'
    elif sf < 25000:
        return '4. Large (10-25K)'
    elif sf < 50000:
        return '5. Major (25-50K)'
    else:
        return '6. Enterprise (50K+)'

df['size_category'] = df['leasedSF'].apply(size_category)

# Size distribution by year
size_year = df.groupby(['year', 'size_category']).size().reset_index(name='count')
size_year_pivot = size_year.pivot(index='year', columns='size_category', values='count').fillna(0)

# Normalize to percentages
size_year_pct = size_year_pivot.div(size_year_pivot.sum(axis=1), axis=0) * 100

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Stacked area
ax1 = axes[0]
size_year_pct.plot(kind='area', stacked=True, ax=ax1, colormap='RdYlGn_r', alpha=0.8)
ax1.axvline(2020, color='black', linestyle='--', linewidth=2)
ax1.set_ylabel('Share of Leases (%)')
ax1.set_xlabel('Year')
ax1.set_title('Lease Size Distribution Over Time')
ax1.legend(title='Size Category', bbox_to_anchor=(1.02, 1), loc='upper left')

# Small vs Large trend
ax2 = axes[1]
small = size_year_pct[['1. Micro (<2.5K)', '2. Small (2.5-5K)', '3. Medium (5-10K)']].sum(axis=1)
large = size_year_pct[['5. Major (25-50K)', '6. Enterprise (50K+)']].sum(axis=1)

ax2.plot(small.index, small.values, 'b-o', linewidth=2, markersize=8, label='Small+Medium (<10K SF)')
ax2.plot(large.index, large.values, 'r-s', linewidth=2, markersize=8, label='Major+Enterprise (>25K SF)')
ax2.axvline(2020, color='gray', linestyle='--', alpha=0.5)
ax2.set_ylabel('Share of Leases (%)')
ax2.set_xlabel('Year')
ax2.set_title('Small vs Large Lease Trend')
ax2.legend()

plt.tight_layout()
plt.savefig('../outputs/lease_fragmentation.png', dpi=150, bbox_inches='tight')
plt.show()

# Quantify the shift
print("\nFragmentation Analysis:")
pre_small = small[small.index.isin([2018, 2019])].mean()
post_small = small[small.index.isin([2022, 2023, 2024])].mean()
print(f"Small leases (<10K): {pre_small:.1f}% (pre) → {post_small:.1f}% (post) | Change: {post_small-pre_small:+.1f}pp")

pre_large = large[large.index.isin([2018, 2019])].mean()
post_large = large[large.index.isin([2022, 2023, 2024])].mean()
print(f"Large leases (>25K): {pre_large:.1f}% (pre) → {post_large:.1f}% (post) | Change: {post_large-pre_large:+.1f}pp")

---

## 7. Market-Level Analysis: Which Cities Are Winning?

**Key Question**: Which markets recovered fastest post-pandemic?

In [ ]:
# Top markets by total SF
top_markets = df.groupby('market')['leasedSF'].sum().nlargest(15).index.tolist()

# Market recovery analysis
market_period = df[df['market'].isin(top_markets)].groupby(['market', 'period']).agg({
    'leasedSF': 'sum'
}).reset_index()

market_pivot = market_period.pivot(index='market', columns='period', values='leasedSF').fillna(0)

# Calculate recovery rate
if 'Pre-Pandemic (2018-2019)' in market_pivot.columns and 'Recovery Year 2 (2023)' in market_pivot.columns:
    market_pivot['recovery_rate'] = (
        market_pivot['Recovery Year 2 (2023)'] / (market_pivot['Pre-Pandemic (2018-2019)']/2)
    ) * 100
    market_pivot = market_pivot.sort_values('recovery_rate', ascending=False)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(16, 8))

# Recovery rate
ax1 = axes[0]
colors = ['green' if x >= 100 else 'red' for x in market_pivot['recovery_rate']]
market_pivot['recovery_rate'].plot(kind='barh', ax=ax1, color=colors)
ax1.axvline(100, color='black', linestyle='--', linewidth=2, label='Pre-pandemic level')
ax1.set_xlabel('Recovery Rate (% of Pre-Pandemic Annual Average)')
ax1.set_title('Market Recovery: 2023 vs Pre-Pandemic')
ax1.legend()

# Heatmap of period changes
ax2 = axes[1]
period_cols = [c for c in market_pivot.columns if 'Pandemic' in c or 'Recovery' in c or 'Pre' in c]
heatmap_data = (market_pivot[period_cols[:4]] / 1e6).round(1)
sns.heatmap(heatmap_data, annot=True, fmt='.1f', cmap='RdYlGn', ax=ax2, cbar_kws={'label': 'SF (Millions)'})
ax2.set_title('Leasing Volume by Market and Period (SF in Millions)')

plt.tight_layout()
plt.savefig('../outputs/market_recovery.png', dpi=150, bbox_inches='tight')
plt.show()

---

## 8. Transaction Type Analysis

**Key Question**: Are companies expanding, relocating, or signing new leases?

In [ ]:
# Transaction type by year
trans_year = df.groupby(['year', 'transaction_type']).agg({
    'leasedSF': ['sum', 'count']
}).reset_index()
trans_year.columns = ['year', 'transaction_type', 'total_sf', 'num_leases']

# Normalize by year
year_totals = trans_year.groupby('year')['num_leases'].sum().reset_index(name='year_total')
trans_year = trans_year.merge(year_totals, on='year')
trans_year['share'] = trans_year['num_leases'] / trans_year['year_total'] * 100

# Pivot
trans_pivot = trans_year.pivot(index='year', columns='transaction_type', values='share').fillna(0)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

ax1 = axes[0]
trans_pivot.plot(kind='area', stacked=True, ax=ax1, colormap='Set2', alpha=0.8)
ax1.axvline(2020, color='black', linestyle='--', linewidth=2)
ax1.set_ylabel('Share of Transactions (%)')
ax1.set_xlabel('Year')
ax1.set_title('Transaction Type Mix Over Time')
ax1.legend(title='Type', bbox_to_anchor=(1.02, 1), loc='upper left')

ax2 = axes[1]
# Focus on Expansion rate as indicator of business confidence
if 'Expansion' in trans_pivot.columns:
    ax2.plot(trans_pivot.index, trans_pivot['Expansion'], 'g-o', linewidth=2, markersize=8, label='Expansion')
if 'New' in trans_pivot.columns:
    ax2.plot(trans_pivot.index, trans_pivot['New'], 'b-s', linewidth=2, markersize=8, label='New')
if 'Relocation' in trans_pivot.columns:
    ax2.plot(trans_pivot.index, trans_pivot['Relocation'], 'r-^', linewidth=2, markersize=8, label='Relocation')
ax2.axvline(2020, color='gray', linestyle='--', alpha=0.5)
ax2.set_ylabel('Share of Transactions (%)')
ax2.set_xlabel('Year')
ax2.set_title('Key Transaction Types Trend')
ax2.legend()

plt.tight_layout()
plt.savefig('../outputs/transaction_types.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nTransaction Type Shifts:")
for ttype in ['New', 'Expansion', 'Relocation']:
    if ttype in trans_pivot.columns:
        pre = trans_pivot.loc[[2018, 2019], ttype].mean()
        post = trans_pivot.loc[[2022, 2023, 2024], ttype].mean() if 2024 in trans_pivot.index else trans_pivot.loc[[2022, 2023], ttype].mean()
        print(f"{ttype}: {pre:.1f}% (pre) → {post:.1f}% (post) | Change: {post-pre:+.1f}pp")

---

## 9. Key Findings & Recommendations

### Summary of Insights

In [ ]:
# Create summary dashboard
fig = plt.figure(figsize=(16, 12))

# Title
fig.suptitle('DataFest 2025: Post-Pandemic Office Market Insights', fontsize=16, fontweight='bold', y=0.98)

# Key metrics
ax1 = fig.add_subplot(2, 3, 1)
ax1.text(0.5, 0.8, 'Pandemic Impact', ha='center', fontsize=14, fontweight='bold')
ax1.text(0.5, 0.5, f'{(pandemic_annual_sf/pre_annual_sf - 1)*100:.0f}%', ha='center', fontsize=36, 
         color='red' if pandemic_annual_sf < pre_annual_sf else 'green')
ax1.text(0.5, 0.2, 'Change in Annual Leasing\n(2020-21 vs 2018-19)', ha='center', fontsize=10)
ax1.axis('off')

ax2 = fig.add_subplot(2, 3, 2)
ax2.text(0.5, 0.8, 'Recovery Status', ha='center', fontsize=14, fontweight='bold')
recovery_pct = (recovery_annual_sf/pre_annual_sf)*100
ax2.text(0.5, 0.5, f'{recovery_pct:.0f}%', ha='center', fontsize=36, 
         color='green' if recovery_pct >= 90 else 'orange')
ax2.text(0.5, 0.2, 'of Pre-Pandemic Level\n(2022+ vs 2018-19)', ha='center', fontsize=10)
ax2.axis('off')

ax3 = fig.add_subplot(2, 3, 3)
ax3.text(0.5, 0.8, 'Class A Trend', ha='center', fontsize=14, fontweight='bold')
class_a_change = post_a - pre_a
ax3.text(0.5, 0.5, f'{class_a_change:+.1f}pp', ha='center', fontsize=36, 
         color='green' if class_a_change > 0 else 'red')
ax3.text(0.5, 0.2, 'Change in Class A Share\n(Flight to Quality)', ha='center', fontsize=10)
ax3.axis('off')

# Time series
ax4 = fig.add_subplot(2, 3, (4, 5))
ax4.plot(quarterly['date'], quarterly['total_sf'] / 1e6, 'b-', linewidth=2)
ax4.fill_between(quarterly['date'], 0, quarterly['total_sf'] / 1e6, alpha=0.3)
ax4.axvline(pd.Timestamp('2020-03-01'), color='red', linestyle='--', alpha=0.7, label='COVID-19 Onset')
ax4.set_ylabel('Total SF (Millions)')
ax4.set_title('Quarterly Leasing Volume')
ax4.legend()

# Top industries
ax5 = fig.add_subplot(2, 3, 6)
top_5 = df.groupby('internal_industry')['leasedSF'].sum().nlargest(5)
(top_5 / 1e6).plot(kind='barh', ax=ax5, color='steelblue')
ax5.set_xlabel('Total SF (Millions)')
ax5.set_title('Top 5 Industries by Total Leasing')

plt.tight_layout()
plt.savefig('../outputs/executive_summary.png', dpi=150, bbox_inches='tight')
plt.show()

---

## 10. Strategic Recommendations for Savills Clients

Based on our analysis, here are actionable recommendations:

### For Tenants (Buyers):

1. **Leverage the Tenant-Favorable Market**
   - Vacancy remains elevated, giving tenants negotiating power
   - Consider longer lease terms in exchange for better rates

2. **Consider Class A Properties**
   - Flight to quality is real - better buildings attract talent
   - Premium pricing may be offset by productivity gains

3. **Right-size Your Space**
   - Hybrid work means less SF needed per employee
   - Consider flexible space options for growth

4. **Industry-Specific Advice**:
   - **Tech**: Recovery strong, consider growth-oriented leases
   - **Finance**: Stable, focus on premium locations
   - **Legal**: Traditional space still valued, CBD preferred

### For Landlords (Sellers):

1. **Invest in Amenities**
   - Buildings with better amenities are winning tenants
   - Consider adding fitness, dining, collaboration spaces

2. **Flexible Terms**
   - Shorter initial terms with renewal options
   - Offer expansion/contraction rights

3. **Target Growth Industries**
   - Focus on tech, healthcare, and professional services
   - Be cautious with industries slow to return to office